In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

# 1. 기본 레이어 계층 

## 레이어 계층 정의

In [3]:
# tf.keras.layers 패키지에서 층은 객체입니다. 층을 구성하려면 간단히 객체를 생성하십시오.
# 대부분의 layer는 첫번째 인수로 출력 차원(크기) 또는 채널을 취합니다.
layer = tf.keras.layers.Dense(100)

In [4]:
layer.variables

[]

In [5]:
# 입력 차원의 수는 층을 처음 실행할 때 유추할 수 있기 때문에 종종 불필요합니다. 
# 일부 복잡한 모델에서는 수동으로 입력 차원의 수를 제공하는것이 유용할 수 있습니다.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [6]:
layer(tf.zeros([10, 5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

## layer.variables`를 사용하여 층안에 있는 모든 변수를 확인

In [7]:
type(layer.variables)

list

In [8]:
len(layer.variables)

2

### 완전 연결(fully-connected)층은 가중치(weight)와 편향(biases)을 위한 변수

In [9]:
layer.kernel.shape

TensorShape([5, 10])

In [10]:
layer.bias.shape

TensorShape([10])

# 2. 사용자 정의 층 구현

In [11]:
help(tf.keras.layers.Layer)

Help on class Layer in module tensorflow.python.keras.engine.base_layer:

class Layer(tensorflow.python.module.module.Module)
 |  Layer(trainable=True, name=None, dtype=None, dynamic=False, **kwargs)
 |  
 |  Base layer class.
 |  
 |  This is the class from which all layers inherit.
 |  
 |  A layer is a class implementing common neural networks operations, such
 |  as convolution, batch norm, etc. These operations require managing weights,
 |  losses, updates, and inter-layer connectivity.
 |  
 |  Users will just instantiate a layer and then treat it as a callable.
 |  
 |  We recommend that descendants of `Layer` implement the following methods:
 |  
 |  * `__init__()`: Save configuration in member variables
 |  * `build()`: Called once from `__call__`, when we know the shapes of inputs
 |    and `dtype`. Should have the calls to `add_weight()`, and then
 |    call the super's `build()` (which sets `self.built = True`, which is
 |    nice in case the user wants to call `build()` ma

In [12]:
class MyDenseLayer(tf.keras.layers.Layer):
    
    ## 레이어 객체 초기화 : 출력 노드 개수
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    ## build ()`: 입력의 모양을 알 때`__call__`에서 한 번 호출
    def build(self, input_shape):
        print(" build once")
        self.kernel = self.add_variable("kernel", shape=[int(input_shape[-1]),self.num_outputs])

    
    ## call ()`:  전달된 인자와 계산
    def call(self, input_):
        print(" call ")
        return tf.matmul(input_, self.kernel)
    
    def __call__(self, *args) :
        print(" __call__")
        return super(MyDenseLayer, self).__call__(*args)

In [13]:
layer = MyDenseLayer(10)


In [14]:
### 내부에 정의된 call 호출

In [15]:
print(layer(tf.zeros([10, 5])))

 __call__
 build once
Instructions for updating:
Please use `layer.add_weight` method instead.
 call 
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)


In [16]:
print(layer.trainable_variables)

[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.1336188 , -0.17973721,  0.0457387 , -0.04778916, -0.11442018,
         0.27634388,  0.0233438 , -0.3605604 , -0.34608147,  0.20435506],
       [ 0.53911453, -0.34215686, -0.0055365 , -0.605517  , -0.1873878 ,
         0.45381778,  0.09408009, -0.17390874, -0.4215914 ,  0.25666666],
       [-0.05356091, -0.2760215 ,  0.16786784,  0.21118021,  0.26238412,
         0.3229065 , -0.12998599,  0.13953787,  0.359532  ,  0.25377226],
       [-0.5885587 ,  0.4719141 ,  0.44792145,  0.08917654, -0.60364455,
        -0.13412514, -0.4745487 ,  0.0325644 , -0.20035127,  0.35161   ],
       [-0.31554815,  0.54240304,  0.25937223,  0.40635914,  0.23181874,
        -0.49638626,  0.34307122,  0.19238013, -0.32875502,  0.1050756 ]],
      dtype=float32)>]


# 3. 사용자 모델 만들어보기

In [17]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        print(" call ")
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)



In [18]:
block = ResnetIdentityBlock(1, [1, 2, 3])

In [19]:
print(block(tf.zeros([1, 2, 3, 3])))

 call 
tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)


In [20]:
print([x.name for x in block.trainable_variables])

['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0']


In [21]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

##  Sequential 로 모델 만들기

In [22]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1, padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])

In [23]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________

In [24]:
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=738, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>